In [215]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

# Moneyball

##### Using the most recent complete baseball statistics dataset from Sean Lahman's website, this notebook provides the names and positions of the nine best players based on their on-base percentage(OBP) and salary.  The nine names presented are determined by looking for players with the best OBP,  lowest salary, and most number of games played in a certain position.  

### Reading in all the .csv files

In [233]:
player_info = pd.read_csv('data/master.csv', usecols=[0, 13, 14]).fillna(0)

In [234]:
batting_info = pd.read_csv('data/batting.csv', usecols=[0, 6, 8, 15, 18, 20]).fillna(0)

In [235]:
salary_info = pd.read_csv('data/salaries.csv', usecols=[0, 3, 4]).fillna(0)

In [236]:
fielding = pd.read_csv('data/appearances.csv', usecols=[3, 5, 8, 9, 10, 11, 12, 13, 14, 15, 16]).fillna(0)

### Merging all the DataFrames

In [220]:
all_info = player_info.merge(batting_info, on='playerID')

In [221]:
all_info = all_info.merge(salary_info, on='playerID')

In [222]:
all_info = all_info.merge(fielding, on='playerID')

### Calculating the OBP for each player during each season and adding it to DataFrame

In [223]:
numerator = all_info['H'] + all_info['BB'] + all_info['HBP']
denominator = all_info['AB'] + all_info['BB'] + all_info['HBP'] + all_info['SF']
all_info['OBP'] = numerator / denominator

In [224]:
obp_per_dollar = all_info['OBP'] / all_info['salary']
all_info['OBP_per_dollar'] = obp_per_dollar

### Filtering data to get latest salary for each player who has been paid in the past 5 years and has a valid OBP.

In [225]:
all_info = all_info.sort_values('yearID')

In [226]:
all_info = all_info[all_info.yearID >= 2011]

In [227]:
all_info = all_info.drop_duplicates('playerID', keep='last')

In [228]:
all_info = all_info.drop_duplicates('OBP', keep=False)

### Choosing the best player for the money for each position

In [229]:
def get_best_player(df):
    p_list = ['G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf']
    best = []
    for item in p_list:
        best.append(df[df[item] > 50].sort(['OBP_per_dollar'], ascending=False)[:1])
    return best

In [230]:
best = get_best_player(all_info)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [231]:
best_players = pd.concat(best)

In [232]:
best_players

,playerID,nameFirst,nameLast,AB,H,BB,HBP,SF,yearID,salary,...,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,OBP,OBP_per_dollar
4126654,woodtr01,Travis,Wood,63.0,14.0,3.0,0.0,0.0,2015,5686000,...,0,0,0,0,0,0,0,0,0.257576,4.529999e-08
3771700,towlejr01,J. R.,Towles,40.0,15.0,3.0,1.0,0.0,2011,424000,...,53,0,0,0,0,0,0,0,0.431818,1.018439e-06
2063734,lahaibr01,Bryan,LaHair,59.0,17.0,9.0,0.0,1.0,2012,482500,...,0,58,0,0,0,2,0,34,0.376812,7.809567e-07
3154517,rhymewi01,Will,Rhymes,191.0,58.0,14.0,0.0,1.0,2011,419000,...,0,0,53,0,0,0,0,0,0.349515,8.341636e-07
955994,duffyma01,Matt,Duffy,573.0,169.0,30.0,5.0,2.0,2015,509000,...,0,1,9,134,3,0,0,0,0.334426,6.570260e-07
354717,bogaexa01,Xander,Bogaerts,613.0,196.0,32.0,3.0,3.0,2015,543000,...,0,0,0,0,156,0,0,0,0.354839,6.534783e-07
1490260,guyerbr01,Brandon,Guyer,259.0,69.0,16.0,11.0,1.0,2015,515800,...,0,0,0,0,0,60,23,41,0.334495,6.484970e-07
2872642,pederjo01,Joc,Pederson,28.0,4.0,9.0,0.0,0.0,2015,510000,...,0,0,0,0,0,0,147,0,0.351351,6.889242e-07
3553479,springe01,George,Springer,388.0,107.0,50.0,8.0,3.0,2015,512900,...,0,0,0,0,0,0,10,93,0.367483,7.164814e-07
